In [1]:
from easyroutine import path_to_parents
import pandas as pd

path_to_parents(1)
import os

%load_ext autoreload
%autoreload 2

Changed working directory to: /orfeo/cephfs/home/dssc/francescortu/HistoryRevisionismLLM


In [2]:
human_val = pd.read_csv("data/human_eval/human_val_26092025.csv")
sample = pd.read_csv("data/sample30_v2.csv")

In [2]:
machine_val = pd.read_csv("data/manual_historical/evaluated/all_models_with_score_v4.csv")

In [3]:
# human_val is a dataframe which coloumns that contais question of a human evaluation. especially we have two specific question "How factually corrected is this response? (Compared to the provided descriptions)" and "Briefly explain your choich (1-3). Plesa cite phrases that influenced you." THis questions are two for each of 30 items. The raws are the answer from different human validation. for the first answer we have a number from 1 to 5 followed by a text explanation of that score. the second is a string or nan. Create a code for obtain a clear dictionary with for each item the parsed list of score and description. The key of the dictionary is the item number from 1 to 30. The value is a list of dictionaries with keys "score" and "description".

In [ ]:
import re
import numpy as np

result = {}

for item in range(30):
    # Build column names for score and description
    if item == 0:
        score_col = " How factually correct is this response? (Compared to the provided descriptions)"
        desc_col = "Briefly explain your choice (1–3 sentences). Please cite phrases that influenced you."
    else:
        score_col = f" How factually correct is this response? (Compared to the provided descriptions).{item}"
        desc_col = f"Briefly explain your choice (1–3 sentences). Please cite phrases that influenced you.  .{item}"

    scores = human_val[score_col].tolist()
    descs = (
        human_val[desc_col].tolist()
        if desc_col in human_val.columns
        else [np.nan] * len(human_val)
    )

    score_list = []
    desc_list = []
    for score_raw, desc in zip(scores, descs):
        # Parse score (extract number 1-5)
        if isinstance(score_raw, str):
            m = re.match(r"\s*(\d)", score_raw)
            score = int(m.group(1)) if m else None
        else:
            score = None
        # Description: if not available, try to extract from score_raw after the dash
        if isinstance(desc, str) and desc.strip():
            description = desc.strip()
        elif isinstance(score_raw, str) and "–" in score_raw:
            description = score_raw.split("–", 1)[1].strip()
        else:
            description = None
        score_list.append(score)
        desc_list.append(description)
    result[item + 1] = {"score": score_list, "description": desc_list}

result

In [6]:
# iterate over sample
def get_llm_val(sample):
    llm_val = {}
    inverse_map = {
        4: 1,
        3: 2,
        2: 3,
        1: 4,
    }
    nan_counter = 0
    for idx, row in sample.iterrows():
        #
        if pd.isna(row["score (gpt4o)"]) or pd.isna(row["score (gemma3)"]):
            nan_counter += 1
            continue
        llm_val[idx] = {
            "gpt4o": inverse_map[row["score (gpt4o)"]],
            "gemma3": inverse_map[row["score (gemma3)"]],
        }
    print(f"Skipped {nan_counter} rows due to NaN values.")
    return llm_val

llm_val = get_llm_val(machine_val)

Skipped 3 rows due to NaN values.


In [5]:
len(llm_val)

16547

In [6]:
human_confusion_martrix = []
for human_scores in result.values():
        human_confusion_martrix.append(human_scores["score"])

In [7]:
machine_confusion_matrix = [(x["gpt4o"], x["gemma3"]) for x in llm_val.values()]

In [12]:
print("########### Fleiss Kappa ###########")

import numpy as np
from collections import Counter

def create_weight_matrix(categories, weight_type='linear'):
    """
    Create a weight matrix for weighted kappa calculations.
    
    Parameters:
    categories (array-like): Sorted list/array of categories
    weight_type (str): 'linear' or 'quadratic' or custom matrix
    
    Returns:
    np.array: Weight matrix where w[i,j] represents disagreement weight
    """
    n_cats = len(categories)
    weights = np.zeros((n_cats, n_cats))
    
    if weight_type == 'linear':
        # Linear weights: |i-j| / (n_cats-1)
        for i in range(n_cats):
            for j in range(n_cats):
                weights[i, j] = 1 - abs(i - j) / (n_cats - 1)
    
    elif weight_type == 'quadratic':
        # Quadratic weights: 1 - [(i-j)/(n_cats-1)]^2
        for i in range(n_cats):
            for j in range(n_cats):
                weights[i, j] = 1 - ((i - j) / (n_cats - 1)) ** 2
    
    elif isinstance(weight_type, np.ndarray):
        # Custom weight matrix provided
        weights = weight_type.copy()
    
    return weights

def weighted_fleiss_kappa(ratings, weight_type='linear', categories=None):
    """
    Calculate weighted Fleiss' kappa for inter-rater reliability.
    
    Parameters:
    ratings (list of lists): Each inner list contains ratings from different raters for one item
    weight_type (str or np.array): 'linear', 'quadratic', or custom weight matrix
    categories (list): Optional ordered list of categories. If None, will be inferred and sorted
    
    Returns:
    float: Weighted Fleiss' kappa coefficient
    """
    ratings = np.array(ratings)
    n_items, n_raters = ratings.shape
    
    # Handle categories
    if categories is None:
        categories = sorted(np.unique(ratings.flatten()))
    else:
        categories = list(categories)
    
    n_categories = len(categories)
    cat_to_idx = {cat: idx for idx, cat in enumerate(categories)}
    
    # Create weight matrix
    if isinstance(weight_type, str):
        weights = create_weight_matrix(categories, weight_type)
    else:
        weights = weight_type
    
    # Create pairing matrix for each item
    pairing_matrices = []
    for item_ratings in ratings:
        # Count occurrences of each category for this item
        counts = np.zeros(n_categories)
        for rating in item_ratings:
            counts[cat_to_idx[rating]] += 1
        
        # Create pairing matrix: how many pairs of (category_i, category_j)
        pairing_matrix = np.outer(counts, counts)
        # Remove self-pairs (same rater can't pair with themselves)
        np.fill_diagonal(pairing_matrix, counts * (counts - 1))
        
        pairing_matrices.append(pairing_matrix)
    
    # Sum all pairing matrices
    total_pairing_matrix = np.sum(pairing_matrices, axis=0)
    
    # Calculate observed weighted agreement
    observed_weighted_agreement = 0
    total_pairs = 0
    
    for i in range(n_categories):
        for j in range(n_categories):
            pairs_ij = total_pairing_matrix[i, j]
            weight_ij = weights[i, j]
            observed_weighted_agreement += pairs_ij * weight_ij
            total_pairs += pairs_ij
    
    P_observed_weighted = observed_weighted_agreement / total_pairs if total_pairs > 0 else 0
    
    # Calculate marginal proportions
    marginal_counts = np.sum(total_pairing_matrix, axis=1)
    total_ratings = np.sum(marginal_counts)
    marginal_proportions = marginal_counts / total_ratings
    
    # Calculate expected weighted agreement
    expected_weighted_agreement = 0
    for i in range(n_categories):
        for j in range(n_categories):
            expected_pairs_ij = marginal_proportions[i] * marginal_proportions[j] * total_pairs
            weight_ij = weights[i, j]
            expected_weighted_agreement += expected_pairs_ij * weight_ij
    
    P_expected_weighted = expected_weighted_agreement / total_pairs if total_pairs > 0 else 0
    
    # Calculate weighted kappa
    if P_expected_weighted == 1.0:
        kappa_weighted = 1.0 if P_observed_weighted == 1.0 else 0.0
    else:
        kappa_weighted = (P_observed_weighted - P_expected_weighted) / (1 - P_expected_weighted)
    
    return kappa_weighted

def weighted_fleiss_kappa_detailed(ratings, weight_type='linear', categories=None):
    """
    Calculate weighted Fleiss' kappa with detailed breakdown.
    
    Returns:
    dict: Dictionary containing weighted kappa and intermediate calculations
    """
    ratings = np.array(ratings)
    n_items, n_raters = ratings.shape
    
    if categories is None:
        categories = sorted(np.unique(ratings.flatten()))
    else:
        categories = list(categories)
    
    n_categories = len(categories)
    cat_to_idx = {cat: idx for idx, cat in enumerate(categories)}
    
    # Create weight matrix
    if isinstance(weight_type, str):
        weights = create_weight_matrix(categories, weight_type)
    else:
        weights = weight_type
    
    # Calculate pairing matrices
    pairing_matrices = []
    for item_ratings in ratings:
        counts = np.zeros(n_categories)
        for rating in item_ratings:
            counts[cat_to_idx[rating]] += 1
        
        pairing_matrix = np.outer(counts, counts)
        np.fill_diagonal(pairing_matrix, counts * (counts - 1))
        pairing_matrices.append(pairing_matrix)
    
    total_pairing_matrix = np.sum(pairing_matrices, axis=0)
    
    # Observed weighted agreement
    observed_weighted_agreement = 0
    total_pairs = 0
    
    for i in range(n_categories):
        for j in range(n_categories):
            pairs_ij = total_pairing_matrix[i, j]
            weight_ij = weights[i, j]
            observed_weighted_agreement += pairs_ij * weight_ij
            total_pairs += pairs_ij
    
    P_observed_weighted = observed_weighted_agreement / total_pairs if total_pairs > 0 else 0
    
    # Marginal proportions and expected agreement
    marginal_counts = np.sum(total_pairing_matrix, axis=1)
    total_ratings = np.sum(marginal_counts)
    marginal_proportions = marginal_counts / total_ratings
    
    expected_weighted_agreement = 0
    for i in range(n_categories):
        for j in range(n_categories):
            expected_pairs_ij = marginal_proportions[i] * marginal_proportions[j] * total_pairs
            weight_ij = weights[i, j]
            expected_weighted_agreement += expected_pairs_ij * weight_ij
    
    P_expected_weighted = expected_weighted_agreement / total_pairs if total_pairs > 0 else 0
    
    # Weighted kappa
    if P_expected_weighted == 1.0:
        kappa_weighted = 1.0 if P_observed_weighted == 1.0 else 0.0
    else:
        kappa_weighted = (P_observed_weighted - P_expected_weighted) / (1 - P_expected_weighted)
    
    return {
        'weighted_kappa': kappa_weighted,
        'P_observed_weighted': P_observed_weighted,
        'P_expected_weighted': P_expected_weighted,
        'weights': weights,
        'categories': categories,
        'pairing_matrix': total_pairing_matrix,
        'marginal_proportions': dict(zip(categories, marginal_proportions)),
        'n_items': n_items,
        'n_raters': n_raters
    }

def fleiss_kappa(ratings):
    """
    Standard (unweighted) Fleiss' kappa - equivalent to weighted with identity weights.
    """
    return weighted_fleiss_kappa(ratings, weight_type=np.eye(len(np.unique(ratings))))

# Example usage and comparisons

    
print("Human Fleiss Kappa:", weighted_fleiss_kappa(machine_confusion_matrix))


print("################ ICC ###################")
def intraclass_correlation(ratings, model='two_way_random'):
    """
    Intraclass Correlation Coefficient (ICC) for continuous or ordinal data.
    
    Models:
    - 'one_way_random': Each item rated by different random raters
    - 'two_way_random': Same raters rate all items, raters are random sample
    - 'two_way_fixed': Same raters rate all items, raters are fixed
    """
    ratings = np.array(ratings, dtype=float)
    n_items, n_raters = ratings.shape
    
    # Calculate means
    grand_mean = np.mean(ratings)
    item_means = np.mean(ratings, axis=1)
    rater_means = np.mean(ratings, axis=0)
    
    # Sum of squares calculations
    # Between items (rows)
    SSB = n_raters * np.sum((item_means - grand_mean) ** 2)
    
    # Between raters (columns) 
    SSW_raters = n_items * np.sum((rater_means - grand_mean) ** 2)
    
    # Within (residual)
    SSW = 0
    for i in range(n_items):
        for j in range(n_raters):
            SSW += (ratings[i, j] - item_means[i] - rater_means[j] + grand_mean) ** 2
    
    # Total sum of squares
    SST = np.sum((ratings - grand_mean) ** 2)
    
    # Mean squares
    MSB = SSB / (n_items - 1) if n_items > 1 else 0
    MSW = SSW / ((n_items - 1) * (n_raters - 1)) if (n_items - 1) * (n_raters - 1) > 0 else 0
    MSW_raters = SSW_raters / (n_raters - 1) if n_raters > 1 else 0
    
    # Calculate ICC based on model
    if model == 'one_way_random':
        # ICC(1,1) - single rater, random raters
        icc = (MSB - MSW) / (MSB + (n_raters - 1) * MSW)
    elif model == 'two_way_random':
        # ICC(2,1) - single rater, random effects
        icc = (MSB - MSW) / (MSB + (n_raters - 1) * MSW + n_raters * (MSW_raters - MSW) / n_items)
    elif model == 'two_way_fixed':
        # ICC(3,1) - single rater, fixed effects
        icc = (MSB - MSW) / (MSB + (n_raters - 1) * MSW)
    
    return max(0, icc)  # ICC should be non-negative

print("Human ICC (two_way_random):", intraclass_correlation
        (machine_confusion_matrix, model='two_way_random'))


def total_agreement(ratings):
    """
    Calculate total agreement proportion among raters.
    
    Parameters:
    ratings (list of lists): Each inner list contains ratings from different raters for one item
    
    Returns:
    float: Proportion of items with complete agreement
    """
    ratings = np.array(ratings)
    n_items, n_raters = ratings.shape
    
    total_agreements = sum(1 for item_ratings in ratings if len(set(item_ratings)) == 1)
    
    return total_agreements / n_items if n_items > 0 else 0
print("Human Total Agreement:", total_agreement(machine_confusion_matrix))

def relaxed_agreement(ratings):
    """
    Calculate relaxed agreement proportion among raters (agree within 1 point).
    
    Parameters:
    ratings (list of lists): Each inner list contains ratings from different raters for one item
    
    Returns:
    float: Proportion of items with relaxed agreement
    """
    ratings = np.array(ratings)
    n_items, n_raters = ratings.shape
    
    relaxed_agreements = sum(
        1 for item_ratings in ratings if max(item_ratings) - min(item_ratings) <= 1
    )
    
    return relaxed_agreements / n_items if n_items > 0 else 0
print("Human Relaxed Agreement:", relaxed_agreement(machine_confusion_matrix))


def correlation_matrix(ratings):
    """
    Calculate the correlation matrix between raters.
    
    Parameters:
    ratings (list of lists): Each inner list contains ratings from different raters for one item
    
    Returns:
    pd.DataFrame: Correlation matrix between raters
    """
    ratings = np.array(ratings)
    n_items, n_raters = ratings.shape
    
    # Create a DataFrame for easier correlation calculation
    df = pd.DataFrame(ratings, columns=[f'Rater_{i+1}' for i in range(n_raters)])
    
    return df.corr()

print("Human Correlation Matrix:\n", correlation_matrix(machine_confusion_matrix))

def pearson_correlation(ratings):
    """
    Calculate the average Pearson correlation between all pairs of raters.
    
    Parameters:
    ratings (list of lists): Each inner list contains ratings from different raters for one item
    
    Returns:
    float: Average Pearson correlation coefficient between raters
    """
    ratings = np.array(ratings)
    n_items, n_raters = ratings.shape
    
    if n_raters < 2:
        return None  # Not enough raters to compute correlation
    
    correlations = []
    for i in range(n_raters):
        for j in range(i + 1, n_raters):
            rater_i = ratings[:, i]
            rater_j = ratings[:, j]
            if np.std(rater_i) == 0 or np.std(rater_j) == 0:
                continue  # Skip if no variance
            corr = np.corrcoef(rater_i, rater_j)[0, 1]
            correlations.append(corr)
    
    return np.mean(correlations) if correlations else None 
print("Human Average Pearson Correlation:", pearson_correlation(machine_confusion_matrix))

########### Fleiss Kappa ###########
Human Fleiss Kappa: 0.3791905609237073
################ ICC ###################
Human ICC (two_way_random): 0.4406932843183705
Human Total Agreement: 0.7074589955267847
Human Relaxed Agreement: 0.9624686329417754
Human Correlation Matrix:
           Rater_1   Rater_2
Rater_1  1.000000  0.489628
Rater_2  0.489628  1.000000
Human Average Pearson Correlation: 0.4896279434545988


In [9]:
table.shape

(30, 3)

In [11]:
AGGREGATION_STRATEGY = "majority_vote"  # mean

human_scores_all = []
gpt4o_scores_all = []
for llm_scores, human_scores in zip(llm_val.values(), result.values()):
    # print(llm_scores, human_scores)

    mean_human = round(np.mean([s for s in human_scores["score"] if s is not None]))
    majority_human = max(set(human_scores["score"]), key=human_scores["score"].count)
    mean_gpt4 = round(
        np.mean([llm_scores["gpt4o"] for s in human_scores["score"] if s is not None])
    )
    majority_gpt4o = max(set([llm_scores["gpt4o"]]), key=[llm_scores["gpt4o"]].count)

    if AGGREGATION_STRATEGY == "majority_vote":
        # if there are no majority, thake the mean and round to the nearest integer
        if len(set(human_scores["score"])) == len(human_scores["score"]):
            human_score = mean_human
        else:
            human_score = majority_human
        if len(set([llm_scores["gpt4o"]])) == 1:
            gpt4o_score = majority_gpt4o
        else:
            gpt4o_score = mean_gpt4

        human_scores_all.append(human_score)
        gpt4o_scores_all.append(gpt4o_score)


# compute correlation
print("############ Choen's kappa ############")
from sklearn.metrics import cohen_kappa_score

print("GPT-4o vs Human:", cohen_kappa_score(human_scores_all, gpt4o_scores_all))

print("############ Weighted Choen's kappa (quadratic) ############")
from sklearn.metrics import cohen_kappa_score
import numpy as np

print(
    "GPT-4o vs Human:",
    cohen_kappa_score(human_scores_all, gpt4o_scores_all, weights="quadratic"),
)

print("############ % Agreement ############")
import numpy as np
print(
    "GPT-4o vs Human:",
    np.mean(np.array(human_scores_all) == np.array(gpt4o_scores_all)),
)

print("############## % Accuracy - within 1 ##############")
import numpy as np
print(
    "GPT-4o vs Human:",
    np.mean(np.abs(np.array(human_scores_all) - np.array(gpt4o_scores_all)) <= 1),
)

print("################# ICC #####################")
confusion_matrix = [[hu,gp] for hu, gp in zip(human_scores_all, gpt4o_scores_all)]
print("GPT-4o vs Human (two_way_random):", intraclass_correlation(
    confusion_matrix, model='two_way_random'
))

print("############ K")

############ Choen's kappa ############
GPT-4o vs Human: 0.24778761061946908
############ Weighted Choen's kappa (quadratic) ############
GPT-4o vs Human: 0.4476744186046512
############ % Agreement ############
GPT-4o vs Human: 0.43333333333333335
############## % Accuracy - within 1 ##############
GPT-4o vs Human: 0.7666666666666667
################# ICC #####################
GPT-4o vs Human (two_way_random): 0.456071076011846
############ K


In [10]:
confusion_matrix

[[3, 4],
 [4, 4],
 [4, 4],
 [2, 4],
 [4, 4],
 [2, 4],
 [4, 4],
 [2, 4],
 [3, 3],
 [3, 3],
 [2, 3],
 [2, 3],
 [2, 3],
 [3, 4],
 [3, 3],
 [2, 3],
 [4, 2],
 [4, 2],
 [2, 2],
 [3, 2],
 [3, 2],
 [2, 2],
 [1, 2],
 [1, 1],
 [1, 1],
 [1, 1],
 [2, 1],
 [1, 1],
 [3, 1],
 [3, 1]]